In [1]:
import pandas as pd
import zipfile
import os
import shutil

In [2]:


# # Convert .pbit to .zip
# pbit_file = 'RBI.pbit'
# zip_file = 'RBI.zip'
# os.rename(pbit_file, zip_file)

# # Extract .zip file
# with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#     extract_folder = 'RBI Extracted'
#     zip_ref.extractall(extract_folder)

# #again renaming app to avoiding loosing
# pbit_file = 'RBI.pbit'
# zip_file = 'RBI.zip'
# os.rename(zip_file,pbit_file)



# load Layout file
import json
filepath = r"RBI Extracted\Report\Layout"
with open(filepath, "r", encoding = 'utf-16 le') as f:
    orgdata = json.load(f)

# Get the list of all files and directories
# path = r"C:\Users\prasadh\Documents\Power BI label extraction and updation using python"
# dir_list = os.listdir(path)
# print("Files and directories in '", path, "' :")
# # prints all files
# print(dir_list)
data = orgdata.copy()


In [3]:
# Update all object labels
excel_file = 'Label_output.xlsx'
exceldata = pd.read_excel(excel_file, sheet_name=None)
position = 0

for (sheet_name, exceldf), jsondata in zip(exceldata.items(), data["sections"]): 
    #pull excel data sheet wise and remove null rows
    exceldf = exceldf[~exceldf["Label"].isna()].reset_index(drop=True)
    # exceldf.set_index("ID",inplace=True)
    # print(exceldf)
    jsonsheetname = jsondata["displayName"] 
    # print(exceldf.index.to_list())

    if(sheet_name == jsonsheetname):
        innerposition = 0
        for j in jsondata["visualContainers"]:

            content = j["config"]
            objectid = content
            objectid = objectid[9:29]
            if(len(exceldf[exceldf["ID"] == objectid])>0):
                NewLabel = list(exceldf[exceldf["ID"] == objectid]["New Label"])[0]
                Label = list(exceldf[exceldf["ID"] == objectid]["Label"])[0]
                # print(Label,NewLabel)
                data["sections"][position]["visualContainers"][innerposition]["config"] = data["sections"][position]["visualContainers"][innerposition]["config"].replace(Label,NewLabel)
            innerposition += 1    
            
        position += 1 


In [4]:
# excel_file = 'Label_output.xlsx'
# # #storing labels sheet wise
# # with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
# count = 0
# for i in data["sections"]:
#         currentsheet = i["displayName"]
#         exceldf = pd.read_excel(excel_file,sheet_name=currentsheet)
#         for j in i["visualContainers"]:
#             content = json.loads(j["config"])     
           
#             objectid = content["name"]
#             newLabel = exceldf[exceldf["ID"] == objectid]["New Label"]
#             if(len(newLabel)>0):
#                 newLabel = newLabel.values[0]
            
#                 try:  
                    
#                     print(content["singleVisual"]["vcObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]["Value"],currentsheet)
#                     content["singleVisual"]["vcObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]["Value"] = newLabel

#                     print(content["singleVisual"]["vcObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]["Value"],currentsheet)

#                     count += 1
#                 except: 
#                     pass

#             content_json_string = json.dumps(content)
#             j["config"] = content_json_string
# print(data)  
# print(count)            

In [5]:
#Label update for all tables
excel_file = 'TableLableExtracted.xlsx'
for i in data["sections"]:
    currentsheet = i["displayName"]

    for j in i["visualContainers"]:
        content = json.loads(j["config"])

        if(content["singleVisual"]["visualType"]=="tableEx"):
            exceldf = pd.read_excel(excel_file,sheet_name=currentsheet)

            for key,value in content["singleVisual"]["columnProperties"].items():
                # print(currentsheet)
                # if(currentsheet == "SSR vs RTD"):
                
                # print(checkid)
                # print(list(exceldf[exceldf["ID"] == checkid]["New Label"])[0] )
                # print(checkid)
                try:
                    checkid = (key+content["name"]+value["displayName"])
                    newvalue = list(exceldf[exceldf["ID"] == checkid]["New Label"])[0]
                    # print(value["displayName"])
                    value["displayName"] = newvalue

                except:
                    # print("here")
                    pass
                    
                
            # print(content)


        content_json_string = json.dumps(content)
        j["config"] = content_json_string
# print(data)
        


In [6]:
#Label update for all bar and donut charts
excel_file = 'BarchartLableExtracted.xlsx'
for i in data["sections"]:
    currentsheet = i["displayName"]

    for j in i["visualContainers"]:
        content = json.loads(j["config"])

        if(content["singleVisual"]["visualType"] in ["clusteredColumnChart","clusteredBarChart","barChart","donutChart"]):
            exceldf = pd.read_excel(excel_file,sheet_name=currentsheet)
            # print(currentsheet)
            for key,value in content["singleVisual"]["columnProperties"].items():
                try:
                    checkid = (key+content["name"]+value["displayName"])
                    newvalue = list(exceldf[exceldf["ID"] == checkid]["New Label"])[0]
                    value["displayName"] = newvalue
                except:
                    pass

        content_json_string = json.dumps(content)
        j["config"] = content_json_string
# print(data)

In [7]:
#dumping with updated labels
with open("RBI Extracted\Report\Layout", 'w', encoding = 'utf-16 le') as outfile:
    json.dump(data, outfile, indent = 4)
#removing security file
  

In [8]:
os.remove(r"RBI Extracted\SecurityBindings")
os.getcwd()  

'c:\\Users\\prasadh\\Documents\\Power BI label extraction and updation using python\\Version 2'

In [9]:
#Repack extracted folder to .zip
extract_folder = 'RBI Extracted'
shutil.make_archive('RBI New App', 'zip', extract_folder)
#Rename .zip to .pbit
new_zip_file = 'RBI New App.zip'
new_pbit_file = 'RBI New App.pbit'
os.rename(new_zip_file, new_pbit_file)

# Clean up extracted folder and original zip file if needed
# shutil.rmtree(extract_folder)
# os.remove(zip_file)


In [10]:
# connection
# ecl-ecosure-tv-sql-001-s.database.windows.net
# TrueViewDWDB